In [47]:
import spacy
from spacy.tokens import Doc
from spacy.matcher import Matcher
from zipfile import ZipFile
from pathlib import Path
from seqeval import scheme
from tqdm import autonotebook as tqdm

import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

import sklearn_crfsuite
from collections import Counter

In [48]:
nlp = spacy.load("en_core_web_sm")

In [49]:
data_dir = Path("./data/teaching-dataset")
with (data_dir / "span_extraction_text_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        sentences = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]
with (data_dir / "span_extraction_references_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("references.txt") as f:
        labels = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]

In [50]:
# idx = 0
# sentence = sentences[idx]
# doc = Doc(nlp.vocab, words=sentence)
# doc = nlp(doc)
# iterator = zip(doc, labels[idx])
# for token, label in iterator:
#     print(f"{token.text :<16} {label :<8} {token.pos_ :<8} {token.dep_ :<8}")

sentence_number=[]
words=[]
tags=[]
dependencies=[]
events=[]
for i in range(len(sentences)):
    doc = Doc(nlp.vocab, words=sentences[i])
    doc = nlp(doc)
    iterator = zip(doc, labels[i])
    for token, label in iterator:
        sentence_number.append(f'Sentence: {i}')
        words.append(str(token))
        tags.append(token.pos_)
        dependencies.append(token.dep_)
        events.append(label)

train_data = {'Sentence #': sentence_number,'Word': words, 'POS': tags, 'DEP': dependencies, 'Tag': events}
df = pd.DataFrame(data=train_data)
print(df)

          Sentence #       Word    POS     DEP      Tag
0        Sentence: 0  According   VERB    prep        O
1        Sentence: 0        two    NUM  nummod        O
2        Sentence: 0  different    ADJ    amod        O
3        Sentence: 0    studies   NOUN    pobj        O
4        Sentence: 0         it   PRON   nsubj        O
...              ...        ...    ...     ...      ...
13425  Sentence: 467         of    ADP    prep        O
13426  Sentence: 467      death   NOUN    pobj  B-EVENT
13427  Sentence: 467        for    ADP    prep        O
13428  Sentence: 467   Haitians  PROPN    pobj        O
13429  Sentence: 467          .  PUNCT   punct        O

[13430 rows x 5 columns]


In [51]:
df.isnull().sum()

df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

df.groupby('Tag').size().reset_index(name='counts')

X = df.drop('Tag', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
X_train.shape, y_train.shape

((10744, 4012), (10744,))

In [52]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 49.92, NNZs: 1467, Bias: -2.000000, T: 10744, Avg. loss: 0.290581
Total training time: 0.09 seconds.
Norm: 69.91, NNZs: 2024, Bias: 3.000000, T: 10744, Avg. loss: 0.573250
Total training time: 0.09 seconds.
Norm: 64.36, NNZs: 1841, Bias: -2.000000, T: 10744, Avg. loss: 0.557241
Total training time: 0.09 seconds.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished


Perceptron(max_iter=5, n_jobs=-1, verbose=10)

In [53]:
new_classes = classes.copy()
print(new_classes)

['B-EVENT', 'I-EVENT', 'O']


In [54]:
print(classification_report(y_pred=per.predict(X_test), y_true=y_test))

              precision    recall  f1-score   support

     B-EVENT       0.32      0.62      0.42       270
     I-EVENT       0.52      0.39      0.45       590
           O       0.86      0.81      0.83      1826

    accuracy                           0.70      2686
   macro avg       0.57      0.61      0.57      2686
weighted avg       0.73      0.70      0.71      2686



In [55]:
sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train, classes)

SGDClassifier()

In [56]:
print(classification_report(y_pred=sgd.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

     B-EVENT       0.34      0.60      0.43       270
     I-EVENT       0.45      0.53      0.49       590
           O       0.89      0.74      0.81      1826

    accuracy                           0.68      2686
   macro avg       0.56      0.62      0.58      2686
weighted avg       0.74      0.68      0.70      2686



In [57]:
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

MultinomialNB(alpha=0.01)

In [58]:
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels = new_classes))

              precision    recall  f1-score   support

     B-EVENT       0.33      0.61      0.43       270
     I-EVENT       0.46      0.37      0.41       590
           O       0.85      0.80      0.82      1826

    accuracy                           0.69      2686
   macro avg       0.55      0.59      0.55      2686
weighted avg       0.71      0.69      0.69      2686



In [59]:
pa =PassiveAggressiveClassifier()
pa.partial_fit(X_train, y_train, classes)

PassiveAggressiveClassifier()

In [60]:
print(classification_report(y_pred=pa.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

     B-EVENT       0.33      0.33      0.33       270
     I-EVENT       0.54      0.38      0.45       590
           O       0.81      0.89      0.85      1826

    accuracy                           0.72      2686
   macro avg       0.56      0.53      0.54      2686
weighted avg       0.70      0.72      0.71      2686



In [61]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, d, t) for w, p, d, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['DEP'].values.tolist(),
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

In [62]:
print(sentences)

[[('According', 'VERB', 'prep', 'O'), ('two', 'NUM', 'nummod', 'O'), ('different', 'ADJ', 'amod', 'O'), ('studies', 'NOUN', 'pobj', 'O'), ('it', 'PRON', 'nsubj', 'O'), ('seems', 'VERB', 'ROOT', 'O'), ('plausible', 'ADJ', 'oprd', 'O'), ('that', 'SCONJ', 'mark', 'O'), ('the', 'DET', 'det', 'O'), ('Pohang', 'PROPN', 'compound', 'B-EVENT'), ('earthquake', 'NOUN', 'nsubjpass', 'I-EVENT'), ('was', 'AUX', 'auxpass', 'O'), ('induced', 'VERB', 'ccomp', 'O'), ('by', 'ADP', 'agent', 'O'), ('EGS', 'PROPN', 'compound', 'B-EVENT'), ('operations', 'NOUN', 'pobj', 'I-EVENT'), ('.', 'PUNCT', 'punct', 'O')], [('Signs', 'NOUN', 'nsubj', 'O'), ('and', 'CCONJ', 'cc', 'O'), ('symptoms', 'NOUN', 'conj', 'O'), ('include', 'VERB', 'ROOT', 'O'), (':', 'PUNCT', 'punct', 'O'), ('Dyspnea', 'PROPN', 'nsubj', 'O'), ('(', 'PUNCT', 'punct', 'O'), ('shortness', 'NOUN', 'appos', 'O'), ('of', 'ADP', 'prep', 'O'), ('breath', 'NOUN', 'pobj', 'O'), (')', 'PUNCT', 'punct', 'O'), ('exacerbated', 'VERB', 'ccomp', 'O'), ('by', 

In [63]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    deptag = sent[i][2]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'deptag': deptag,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        deptag1 = sent[i-1][2]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:deptag': deptag1,
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        deptag1 = sent[i+1][2]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:deptag': deptag1,
        })
    else:
        features['EOS'] = True
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, deptag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, deptag, label in sent]

In [64]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=0)

In [65]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train);

In [66]:
def flatten(l):
    output = []
    for sublist in l:
        for item in sublist:
            output.append(item)

        output.append('')
    return output

y_pred = crf.predict(X_test)
# print(metrics.flat_classification_report(y_pred=y_pred, y_true=y_test, labels = new_classes))
# sollte eigentlich funktionieren tuts aber nicht
flat_y_true = flatten(y_test)
flat_y_pred = flatten(y_pred)
print(classification_report(flat_y_true, flat_y_pred))

              precision    recall  f1-score   support

                   1.00      1.00      1.00         5
     B-EVENT       0.73      0.50      0.59        16
     I-EVENT       0.59      0.59      0.59        32
           O       0.80      0.84      0.82        83

    accuracy                           0.75       136
   macro avg       0.78      0.73      0.75       136
weighted avg       0.75      0.75      0.75       136



In [67]:
data_dir = Path("./data/teaching-dataset")
with (data_dir / "span_extraction_text_test.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        test_sentences = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]

print(len(test_sentences))

98


In [68]:
sentence_number=[]
words=[]
tags=[]
dependencies=[]
events=[]
for i in range(len(test_sentences)):
    doc = Doc(nlp.vocab, words=test_sentences[i])
    doc = nlp(doc)
    for token in doc:
        sentence_number.append(f'Sentence: {i}')
        words.append(str(token))
        tags.append(token.pos_)
        dependencies.append(token.dep_)

train_data = {'Sentence #': sentence_number,'Word': words, 'POS': tags, 'DEP': dependencies}
df = pd.DataFrame(data=train_data)
print(df)

        Sentence #               Word    POS       DEP
0      Sentence: 0              After    ADP      prep
1      Sentence: 0                the    DET       det
2      Sentence: 0            Trinity  PROPN  compound
3      Sentence: 0               test   NOUN      pobj
4      Sentence: 0                  ,  PUNCT     punct
...            ...                ...    ...       ...
2849  Sentence: 97         inhibitory    ADJ      amod
2850  Sentence: 97  neurotransmitters   NOUN      pobj
2851  Sentence: 97               like    ADP      prep
2852  Sentence: 97               GABA  PROPN      pobj
2853  Sentence: 97                  .  PUNCT     punct

[2854 rows x 4 columns]


In [85]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, d) for w, p, d in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['DEP'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        

        print(self.grouped.head(20))
        self.sentences = [s for s in self.grouped]
        
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences
for sentence in sentences:
    print(sentence)
# print(sentences)

Sentence #
0     [(After, ADP, prep), (the, DET, det), (Trinity...
1     [(Arteriovenous, ADJ, amod), (fistula, NOUN, n...
2     [(A, DET, det), (study, NOUN, nsubj), (by, ADP...
3     [(Dietary, ADJ, nsubj), (causes, VERB, ROOT), ...
4     [(On, ADP, prep), (November, PROPN, pobj), (20...
5     [(A, DET, det), (study, NOUN, nsubj), (done, V...
6     [(Phytophthora, NOUN, ROOT), (a, DET, det), (g...
7     [(Pain, NOUN, nsubjpass), (associated, VERB, a...
8     [(During, ADP, prep), (underground, ADJ, amod)...
9     [(Discrediting, VERB, advcl), (earlier, ADJ, a...
10    [(During, ADP, prep), (the, DET, det), (1970s,...
11    [(Late, ADJ, amod), (-, PUNCT, punct), (phase,...
12    [(The, DET, det), (2008, NUM, nummod), (Sichua...
13    [(With, ADP, prep), (interventional, ADJ, amod...
14    [(People, NOUN, nsubj), (who, PRON, nsubj), (h...
15    [(Mutations, NOUN, nsubj), (in, ADP, prep), (t...
16    [(The, DET, det), (undigested, ADJ, amod), (fa...
17    [(Political, ADJ, amod), (insta

In [86]:
X = [sent2features(s) for s in sentences]
y_pred = crf.predict(X)
print(y_pred)
flat_y_pred = flatten(y_pred)

[['O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'O'], ['B-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'B-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT

In [41]:
Path("predictions.txt").write_text("\n".join(map(str, flat_y_pred)));

In [35]:
# evaluation wie auf tira
# evaluate ist ein package, das ihr installieren müsst, `pip install evaluate`
import evaluate

# speichert die training predictions unter predictions.txt ab
# führt dann diese zelle aus. falls alles funktioniert, sollte es auch auf tira gehen
# auf tira aber natürlich den test datensatz verwenden!!!!

loaded_predictions = list(sentence.split("\n") for sentence in Path("predictions.txt").read_text().split("\n\n"))
print(loaded_predictions)

evaluator = evaluate.load("fschlatt/ner_eval")
evaluator.compute(predictions=loaded_predictions, references=labels)

[["['O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']", "['B-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'O']", "['O', 'O', 'O', 'O', 'B-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']", "['B-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'O']", "['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'O', 'O', 'O', 'O', 'O', 'O']", "['O', 'B-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

ValueError: Mismatch in the number of predictions (1) and references (468)